<a href="https://colab.research.google.com/github/yashveersinghsohi/Hands_On_ML_Book_Practice/blob/master/Chapter_7/Practice_Chapter7_Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Packages

In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import (
    train_test_split, 
    RepeatedStratifiedKFold, 
    cross_val_score, 
    RepeatedKFold
  )
from sklearn.datasets import (
    make_moons, 
    load_iris, 
    make_classification, 
    make_regression
  )

from sklearn.ensemble import (
    RandomForestClassifier, 
    VotingClassifier, 
    BaggingClassifier, 
    RandomForestClassifier, 
    AdaBoostClassifier, 
    GradientBoostingRegressor, 
    StackingClassifier, 
    StackingRegressor
  )
from sklearn.linear_model import (
    LogisticRegression, 
    LinearRegression
  )
from sklearn.neighbors import (
    KNeighborsClassifier, 
    KNeighborsRegressor
  )
from sklearn.svm import (
    SVC, 
    SVR
  )
from sklearn.tree import (
    DecisionTreeClassifier, 
    DecisionTreeRegressor
  )
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import (
    accuracy_score, 
    mean_squared_error
  )

import xgboost

# Voting Classifier

In [2]:
X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [3]:
log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

Hard Voting

In [4]:
voting_clf = VotingClassifier(
    estimators=[
                ("lr", log_clf), 
                ("rf", rnd_clf), 
                ("svc", svm_clf)
              ], 
    voting="hard"
  )
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                             

In [5]:
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_true=y_test, y_pred=y_pred))

LogisticRegression 0.864
RandomForestClassifier 0.904
SVC 0.896
VotingClassifier 0.912


Soft voting

In [6]:
log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC(probability=True)

voting_clf = VotingClassifier(
    estimators=[
                ("lr", log_clf), 
                ("rf", rnd_clf), 
                ("svc", svm_clf)
              ], 
    voting="soft"
  )
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                             

In [7]:
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_true=y_test, y_pred=y_pred))

LogisticRegression 0.864
RandomForestClassifier 0.896
SVC 0.896
VotingClassifier 0.904


# Bagging 

In [8]:
X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [9]:
bag_clf = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(), 
    n_estimators=500, 
    max_samples=100, 
    bootstrap=True, 
    n_jobs=-1, 
    oob_score=True
  )
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

In [10]:
bag_clf.oob_score_, accuracy_score(y_true=y_test, y_pred=y_pred)

(0.9226666666666666, 0.92)

In [11]:
bag_clf.oob_decision_function_[:5]

array([[0.28125   , 0.71875   ],
       [0.42216359, 0.57783641],
       [1.        , 0.        ],
       [0.0025974 , 0.9974026 ],
       [0.01333333, 0.98666667]])

# Random Forests

In [12]:
X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

rnd_clf = RandomForestClassifier(
    n_estimators=500, 
    max_leaf_nodes=16, 
    n_jobs=-1
  )
rnd_clf.fit(X_train, y_train)
y_pred = rnd_clf.predict(X_test)
accuracy_score(y_true=y_test, y_pred=y_pred)

0.912

# Feature Importances

In [13]:
iris = load_iris()
X, y = iris.data, iris.target

rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(X, y)

for name, score in zip(iris.feature_names, rnd_clf.feature_importances_):
  print(name, round(score, 4))

sepal length (cm) 0.108
sepal width (cm) 0.0225
petal length (cm) 0.438
petal width (cm) 0.4315


# AdaBoost

In [14]:
X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

ada_clf = AdaBoostClassifier(
    base_estimator=DecisionTreeClassifier(max_depth=1), 
    n_estimators=200, 
    algorithm="SAMME.R", 
    learning_rate=0.5
  )
ada_clf.fit(X_train, y_train)

y_pred = ada_clf.predict(X_test)
accuracy_score(y_true=y_test, y_pred=y_pred)

0.896

# Gradient Boosting

In [15]:
np.random.seed(42)
X = np.random.rand(100, 1) - 0.5
y = 3*X[:, 0]**2 + 0.05 * np.random.randn(100)

In [16]:
gbrt = GradientBoostingRegressor(
    max_depth=2, 
    n_estimators=3, 
    learning_rate=1.0
  )
gbrt.fit(X, y)
np.sqrt(mean_squared_error(y_true=y, y_pred=gbrt.predict(X)))

0.07097928528094154

Gradient Boosting with early stopping

In [17]:
np.random.seed(42)
X = np.random.rand(100, 1) - 0.5
y = 3*X[:, 0]**2 + 0.05 * np.random.randn(100)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42
  )

In [18]:
gbrt = GradientBoostingRegressor(
    max_depth=2, 
    n_estimators=120, 
    random_state=42
  )
gbrt.fit(X_train, y_train)

np.sqrt(
    mean_squared_error(
        y_true=y_test, 
        y_pred=gbrt.predict(X_test)
      )
  )

0.04759227782421127

In [19]:
errors = [mean_squared_error(y_true=y_test, y_pred=y_pred) \
          for y_pred in gbrt.staged_predict(X_test)]
best_n_estimators = np.argmin(errors) + 1

In [20]:
best_n_estimators

94

In [21]:
gbrt = GradientBoostingRegressor(
    max_depth=2, 
    n_estimators=best_n_estimators, 
    random_state=42
  )
gbrt.fit(X_train, y_train)

np.sqrt(
    mean_squared_error(
        y_true=y_test, 
        y_pred=gbrt.predict(X_test)
      )
  )

0.04740912595893978

# XGBoost

In [22]:
np.random.seed(42)
X = np.random.rand(100, 1) - 0.5
y = 3*X[:, 0]**2 + 0.05 * np.random.randn(100)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42
  )

In [23]:
xgb_reg = xgboost.XGBRegressor(random_state=42)
xgb_reg.fit(
    X_train, y_train, 
    eval_set=[(X_test, y_test)], 
    early_stopping_rounds=2
  )

y_pred = xgb_reg.predict(X_test)
np.sqrt(mean_squared_error(y_true=y_test, y_pred=y_pred))

[07:15:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:0.260696
Will train until validation_0-rmse hasn't improved in 2 rounds.
[1]	validation_0-rmse:0.234901
[2]	validation_0-rmse:0.211825
[3]	validation_0-rmse:0.191044
[4]	validation_0-rmse:0.173044
[5]	validation_0-rmse:0.156515
[6]	validation_0-rmse:0.141707
[7]	validation_0-rmse:0.128399
[8]	validation_0-rmse:0.116099
[9]	validation_0-rmse:0.105241
[10]	validation_0-rmse:0.095796
[11]	validation_0-rmse:0.087486
[12]	validation_0-rmse:0.080175
[13]	validation_0-rmse:0.074239
[14]	validation_0-rmse:0.069596
[15]	validation_0-rmse:0.065375
[16]	validation_0-rmse:0.061101
[17]	validation_0-rmse:0.058473
[18]	validation_0-rmse:0.056363
[19]	validation_0-rmse:0.053959
[20]	validation_0-rmse:0.05253
[21]	validation_0-rmse:0.05154
[22]	validation_0-rmse:0.05064
[23]	validation_0-rmse:0.049467
[24]	validation_0-rmse:0.048814
[25]	validation_0-rmse

0.046856680237859674

# Stacking Classifier

In [32]:
X, y = make_classification(
    n_samples=1000, 
    n_features=20, 
    n_informative=15, 
    n_redundant=5, 
    random_state=1
  )
X.shape, y.shape

((1000, 20), (1000,))

In [33]:
models = {
  "lr": LogisticRegression(), 
  "knn": KNeighborsClassifier(), 
  "cart": DecisionTreeClassifier(), 
  "svm": SVC(), 
  "bayes": GaussianNB()
}

In [34]:
results = {}
for model_name, model in models.items():
  cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
  scores = cross_val_score(
      estimator=model, 
      X=X, 
      y=y, 
      scoring="accuracy", 
      cv=cv, 
      error_score="raise", 
      n_jobs=-1
    )
  results[model_name] = scores

In [38]:
for model_name, model_scores in results.items():
  print(model_name, end="\t")
  print(round(np.mean(model_scores), 3), end="\t")
  print(round(np.std(model_scores), 3), end="\n")

lr	0.866	0.029
knn	0.931	0.025
cart	0.82	0.041
svm	0.957	0.02
bayes	0.833	0.031


In [40]:
level0_models = [
  ("lr", LogisticRegression()), 
  ("knn", KNeighborsClassifier()), 
  ("cart", DecisionTreeClassifier()), 
  ("svm", SVC()), 
  ("bayes", GaussianNB())
]

level1_model = LogisticRegression()

stacked_models = StackingClassifier(
    estimators=level0_models, 
    final_estimator=level1_model, 
    cv=5
  )

In [41]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(
    estimator=stacked_models, 
    X=X, 
    y=y, 
    scoring="accuracy", 
    cv=cv, 
    error_score="raise", 
    n_jobs=-1
  )

mean_scores = round(np.mean(scores), 3)
std_scores = round(np.std(scores), 3)
print(f"Stacked Models: {mean_scores}\t{std_scores}")

Stacked Models: 0.963	0.02


# Stacking Regression

In [62]:
X, y = make_regression(
    n_samples=1000, 
    n_features=20, 
    n_informative=15, 
    noise=0.1, 
    random_state=1
  )
X.shape, y.shape

((1000, 20), (1000,))

In [63]:
models = {
  "knn": KNeighborsRegressor(), 
  "cart": DecisionTreeRegressor(), 
  "svm": SVR()
}

In [64]:
results = {}
for model_name, model in models.items():
  cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
  scores = cross_val_score(
      estimator=model, 
      X=X, 
      y=y, 
      scoring="neg_mean_absolute_error", 
      cv=cv, 
      error_score="raise", 
      n_jobs=-1
    )
  results[model_name] = scores

In [65]:
for model_name, model_scores in results.items():
  print(model_name, end="\t")
  print(round(np.mean(model_scores), 3), end="\t")
  print(round(np.std(model_scores), 3), end="\n")

knn	-101.019	7.161
cart	-149.012	11.73
svm	-162.419	12.565


In [66]:
level0_models = [ 
  ("knn", KNeighborsRegressor()), 
  ("cart", DecisionTreeRegressor()), 
  ("svm", SVR())
]

level1_model = LinearRegression()

stacked_models = StackingRegressor(
    estimators=level0_models, 
    final_estimator=level1_model, 
    cv=5
  )

In [67]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(
    estimator=stacked_models, 
    X=X, 
    y=y, 
    scoring="neg_mean_absolute_error", 
    cv=cv, 
    error_score="raise", 
    n_jobs=-1
  )

mean_scores = round(np.mean(scores), 3)
std_scores = round(np.std(scores), 3)
print(f"Stacked Models: {mean_scores}\t{std_scores}")

Stacked Models: -56.617	5.388
